<a href="https://colab.research.google.com/github/MohammadErfanRashidi/Breast-Cancer-NeuralNetwork/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import necessary libraries
from IPython import get_ipython # This imports the get_ipython function, which is used to interact with the IPython kernel.
from IPython.display import display # This imports the display function, which is used to display objects in the notebook.

In [2]:
# Import necessary libraries for building and training the neural network
import torch # This imports the PyTorch library, which is used for deep learning.
import torch.nn as nn # This imports the neural network module from PyTorch.
import torch.optim as optim # This imports the optimization module from PyTorch.
from sklearn.datasets import load_breast_cancer # This imports the load_breast_cancer function from scikit-learn, which is used to load the breast cancer dataset.
from sklearn.model_selection import train_test_split # This imports the train_test_split function from scikit-learn, which is used to split the data into training and testing sets.
from sklearn.preprocessing import StandardScaler # This imports the StandardScaler class from scikit-learn, which is used to standardize the data.

In [3]:
# Set the device to use for training (GPU if available, otherwise CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # This creates a torch.device object, which represents the device to use for training.
print(f"Using device: {device}") # This prints the device that is being used.

Using device: cuda


In [4]:
# Load the breast cancer dataset
data = load_breast_cancer() # This loads the breast cancer dataset into a variable called data.
X = data.data # This extracts the features from the dataset and assigns them to a variable called X.
y = data.target # This extracts the target variable from the dataset and assigns it to a variable called y.

In [5]:
# Print the features
print(X) # This prints the features.

[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]


In [6]:
# Print the first 5 target values
print(y[:5]) # This prints the first 5 target values.

[0 0 0 0 0]


In [7]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # This splits the data into training and testing sets, with 20% of the data being used for testing.

In [8]:
# Print the shapes of the data
print(X.shape) # This prints the shape of the original data.
print(X_train.shape) # This prints the shape of the training data.
print(X_test.shape) # This prints the shape of the testing data.

(569, 30)
(455, 30)
(114, 30)


In [9]:
# Standardize the data
scaler = StandardScaler() # This creates a StandardScaler object.

X_train = scaler.fit_transform(X_train) # This fits the StandardScaler to the training data and transforms it.
X_test = scaler.transform(X_test) # This transforms the testing data using the fitted StandardScaler.

In [10]:
# Print the type of X_train
type(X_train) # This prints the type of X_train.

numpy.ndarray

In [11]:
# Convert the data to PyTorch tensors and move to the device
X_train = torch.tensor(X_train, dtype=torch.float32).to(device) # This converts the training data to a PyTorch tensor and moves it to the device.
y_train = torch.tensor(y_train, dtype=torch.float32).to(device) # This converts the training labels to a PyTorch tensor and moves it to the device.
X_test = torch.tensor(X_test, dtype=torch.float32).to(device) # This converts the testing data to a PyTorch tensor and moves it to the device.
y_test = torch.tensor(y_test, dtype=torch.float32).to(device) # This converts the testing labels to a PyTorch tensor and moves it to the device.

In [12]:
# Define the neural network architecture
class NeuralNet(nn.Module): # This defines a class called NeuralNet, which inherits from nn.Module.

  def __init__(self, input_size, hidden_size, output_size): # This is the constructor of the class.
    super(NeuralNet, self).__init__() # This calls the constructor of the parent class.
    self.fc1 = nn.Linear(input_size, hidden_size) # This creates a linear layer with input_size inputs and hidden_size outputs.
    self.relu = nn.ReLU() # This creates a ReLU activation function.
    self.fc2 = nn.Linear(hidden_size, output_size) # This creates a linear layer with hidden_size inputs and output_size outputs.
    self.sigmoid = nn.Sigmoid() # This creates a sigmoid activation function.

  def forward(self, x): # This is the forward pass of the network.
    out = self.fc1(x) # This applies the first linear layer to the input.
    out = self.relu(out) # This applies the ReLU activation function to the output of the first linear layer.
    out = self.fc2(out) # This applies the second linear layer to the output of the ReLU activation function.
    out = self.sigmoid(out) # This applies the sigmoid activation function to the output of the second linear layer.
    return out # This returns the output of the network.

In [13]:
# Set the hyperparameters
input_size = X_train.shape[1] # This sets the input size to the number of features in the training data.
hidden_size = 64 # This sets the hidden size to 64.
output_size = 1 # This sets the output size to 1.
learning_rate = 0.001 # This sets the learning rate to 0.001.
num_epochs = 100 # This sets the number of epochs to 100.

In [14]:
# Create an instance of the neural network
model = NeuralNet(input_size, hidden_size, output_size).to(device) # This creates an instance of the NeuralNet class and moves it to the device.

In [15]:
# Define the loss function and optimizer
criterion = nn.BCELoss() # This creates a binary cross-entropy loss function.
optimizer = optim.Adam(model.parameters(), lr=learning_rate) # This creates an Adam optimizer.

In [16]:
# Train the model
for epoch in range(num_epochs): # This loops over the number of epochs.
  model.train() # This sets the model to training mode.
  optimizer.zero_grad() # This sets the gradients to zero.
  outputs = model(X_train) # This passes the training data through the model.
  loss = criterion(outputs, y_train.view(-1,1)) # This calculates the loss.
  loss.backward() # This calculates the gradients.
  optimizer.step() # This updates the model's parameters.

  with torch.no_grad(): # This disables gradient calculation.
    predicted = outputs.round() # This rounds the predictions to 0 or 1.
    correct = (predicted == y_train.view(-1,1)).float().sum() # This calculates the number of correct predictions.
    accuracy = correct/y_train.size(0) # This calculates the accuracy.

  if (epoch+1) % 10 == 0: # This prints the loss and accuracy every 10 epochs.
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss : {loss.item():.4f}, Accuracy: {accuracy.item() * 100:.2f}%")

Epoch [10/100], Loss : 0.5793, Accuracy: 89.67%
Epoch [20/100], Loss : 0.4583, Accuracy: 92.09%
Epoch [30/100], Loss : 0.3642, Accuracy: 93.41%
Epoch [40/100], Loss : 0.2909, Accuracy: 94.51%
Epoch [50/100], Loss : 0.2356, Accuracy: 94.95%
Epoch [60/100], Loss : 0.1954, Accuracy: 95.60%
Epoch [70/100], Loss : 0.1666, Accuracy: 95.82%
Epoch [80/100], Loss : 0.1455, Accuracy: 96.04%
Epoch [90/100], Loss : 0.1296, Accuracy: 97.14%
Epoch [100/100], Loss : 0.1172, Accuracy: 97.36%


In [17]:
# Evaluate the model on the training data
model.eval() # This sets the model to evaluation mode.
with torch.no_grad(): # This disables gradient calculation.
  outputs = model(X_train) # This passes the training data through the model.
  predicted = outputs.round() # This rounds the predictions to 0 or 1.
  correct = (predicted == y_train.view(-1,1)).float().sum() # This calculates the number of correct predictions.
  accuracy = correct/y_train.size(0) # This calculates the accuracy.
  print(f"Accuracy on training data: {accuracy.item() * 100:.2f}%")

Accuracy on training data: 97.36%


In [18]:
# Evaluate the model on the testing data
model.eval() # This sets the model to evaluation mode.
with torch.no_grad(): # This disables gradient calculation.
  outputs = model(X_test) # This passes the testing data through the model.
  predicted = outputs.round() # This rounds the predictions to 0 or 1.
  correct = (predicted == y_test.view(-1,1)).float().sum() # This calculates the number of correct predictions.
  accuracy = correct/y_test.size(0) # This calculates the accuracy.
  print(f"Accuracy on test data: {accuracy.item() * 100:.2f}%")

Accuracy on test data: 98.25%
